In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd   

# Pobranie informacji o spółkach

In [2]:
url = 'https://www.bankier.pl/gielda/notowania/new-connect'
page = requests.get(url)
soup = BeautifulSoup(page.content,'lxml')
table1 = soup.find_all('table')[0]


#Wczytaj tabele jako dataframe
df = pd.read_html(str(table1))[0]
with open('Pozyskane_dane/Notowania_spolek.csv', 'a') as f:
            df.to_csv(f, header=True,index=False)

# Przygotowanie danych

In [4]:
df1 = pd.read_csv('Pozyskane_dane/Notowania_spolek.csv',encoding= 'unicode_escape', skipinitialspace=True)
df1.columns = ['Walor', 'Kurs', 'Zmiana','Zmiana_procentowa','Liczba_transakcji','Obrot','Otwarcie','Max','Min','Czas']

df1["Obrot"] = df1["Obrot"].astype(str).str.replace('\D+', '') #zamiana na stringi i usunięcie spacji między cyframi
df1["Obrot"] = pd.to_numeric(df1["Obrot"], errors='coerce') #zamiana kolumny Obrot na float
spolki = df1.sort_values(by=['Obrot'],ascending=False).head(30) #30 spolek o najwiekszych obrotach w dniu: xx.xx.xxxx

spolka = []
for i in spolki["Walor"]:
    spolka.append(i)

# Pobranie postów spółek z największymi obrotami

In [ ]:
url2 = 'https://www.bankier.pl/forum/forum_gielda,6,1.html'
page2 = requests.get(url2)
soup2 = BeautifulSoup(page2.content,'lxml')
table2 = soup2.find_all('fieldset')[1] #pobranie z pola wyboru nazwy spolki oraz dopisku do url

#porownanie spolek z listy i pobieranie postow z forum
for i in spolka:
    for option in table2.find_all('option'):
        if option.text == i:
            for x in range(1, 35):
                url = 'https://www.bankier.pl/forum/forum_o_{},{}.html'.format(option['value'], x)
                page = requests.get(url)
                soup = BeautifulSoup(page.content,'lxml')
                try:
                    table = soup.find_all('table')[0]
                    df = pd.read_html(str(table))[0]
                    df['Spolka'] = i

                    with open('Pozyskane_dane/Posty_wybranych_spolek.csv', 'a') as f:
                        df.to_csv(f, header=False)

                except:
                    pass